In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

# 定義CNN模型
def create_cnn_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax')) # 10個類別的輸出層
    return model

# 載入資料集
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 預處理資料
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

# 創建模型
model = create_cnn_model()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 訓練模型
model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split=0.2)

# 評估模型
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

Epoch 1/5


C:\Users\liner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


750/750 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.8542 - loss: 0.4810 - val_accuracy: 0.9779 - val_loss: 0.0704
Epoch 2/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9829 - loss: 0.0557 - val_accuracy: 0.9830 - val_loss: 0.0564
Epoch 3/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9879 - loss: 0.0391 - val_accuracy: 0.9881 - val_loss: 0.0400
Epoch 4/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9911 - loss: 0.0278 - val_accuracy: 0.9848 - val_loss: 0.0497
Epoch 5/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9927 - loss: 0.0234 - val_accuracy: 0.9904 - val_loss: 0.0379
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9863 - loss: 0.0448
Test accuracy: 0.9886000156402588


Store Images In Sql

In [10]:
import csv
import json
import mysql.connector
import pandas as pd
import sqlalchemy as sql
from sqlalchemy import text
import numpy as np


with open('final_config.json') as f:
    config = json.load(f)
host = config['host']
user = config['user']
passwd = config['passwd']

In [8]:
cnx = mysql.connector.connect(
    host=host,
    user=user,
    password=passwd,
    database='my_database'
)

In [5]:
# Create a cursor object
cur = cnx.cursor()

# Create a table for storing images
# id SERIAL PRIMARY KEY,
# create_table_query = """
# CREATE TABLE images (
#     id INT AUTO_INCREMENT PRIMARY KEY,
#     data text
# );
# """
create_table_query = """
CREATE TABLE input_image (
	image_index INT,
    i INT,
    j INT,
    value float,
    CHECK (value >= 0 AND value <= 1)
);
"""
cur.execute(create_table_query)
cnx.commit()

In [6]:
# Convert the NumPy array to a list and then to a JSON string
# for image_data in train_images:
#     # Convert the NumPy array to a list and then to a JSON string
#     train_images_list = image_data.tolist()
#     train_images_json = json.dumps(train_images_list)
#     image_300 = train_images[299]
#     # Insert the JSON string into the database
#     cur.execute("INSERT INTO images (data) VALUES (%s)",(train_images_json,))
#     cnx.commit()
for image_index in range(60000):
    image_data = train_images[image_index]
    for i in range(28):
        for j in range(28):
            value = float(image_data[i, j, 0])
            if value == 0:
                continue
            cur.execute("INSERT INTO input_image (image_index, i, j, value) VALUES (%s, %s, %s, %s)", 
                           (image_index+1, i, j, value))

    # 提交事务每10张图像
    if (image_index+1) % 1000 == 0:
        cnx.commit()
        print(f'{image_index + 1} images inserted')

# 提交剩余的事务
cnx.commit()
print('All images inserted')

# Close the cursor and connection
cur.close()
cnx.close()

1000 images inserted
2000 images inserted
3000 images inserted
4000 images inserted
5000 images inserted
6000 images inserted
7000 images inserted
8000 images inserted
9000 images inserted
10000 images inserted
11000 images inserted
12000 images inserted
13000 images inserted
14000 images inserted
15000 images inserted
16000 images inserted
17000 images inserted
18000 images inserted
19000 images inserted
20000 images inserted
21000 images inserted
22000 images inserted
23000 images inserted
24000 images inserted
25000 images inserted
26000 images inserted
27000 images inserted
28000 images inserted
29000 images inserted
30000 images inserted
31000 images inserted
32000 images inserted
33000 images inserted
34000 images inserted
35000 images inserted
36000 images inserted
37000 images inserted
38000 images inserted
39000 images inserted
40000 images inserted
41000 images inserted
42000 images inserted
43000 images inserted
44000 images inserted
45000 images inserted
46000 images insert

In [11]:
cur = cnx.cursor()
create_table_query = """
CREATE TABLE test_image (
	image_index INT,
    i INT,
    j INT,
    value float,
    CHECK (value >= 0 AND value <= 1)
);
"""
cur.execute(create_table_query)
cnx.commit()

In [12]:
for image_index in range(10000):
    image_data = train_images[image_index]
    for i in range(28):
        for j in range(28):
            value = float(image_data[i, j, 0])
            if value == 0:
                continue
            cur.execute("INSERT INTO test_image (image_index, i, j, value) VALUES (%s, %s, %s, %s)", 
                           (image_index+1, i, j, value))

    # 提交事务每10张图像
    if (image_index+1) % 1000 == 0:
        cnx.commit()
        print(f'{image_index + 1} images inserted')

# 提交剩余的事务
cnx.commit()
print('All test images inserted!')

# Close the cursor and connection
cur.close()
cnx.close()

1000 images inserted
2000 images inserted
3000 images inserted
4000 images inserted
5000 images inserted
6000 images inserted
7000 images inserted
8000 images inserted
9000 images inserted
10000 images inserted
All test images inserted!
